In [2]:
import pandas as pd
complaint=pd.read_csv('shared/complaints_25Nov21.csv')

In [3]:
complaint

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2016-10-26,Money transfers,International money transfer,Other transaction issues,NaN,"To whom it concerns, I would like to file a fo...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",NaN,NaN,NaN,Consent provided,Web,2016-10-29,Closed with explanation,Yes,No,2180490
1,2015-03-27,Bank account or service,Other bank product/service,"Account opening, closing, or management",NaN,My name is XXXX XXXX XXXX and huband name is X...,Company chooses not to provide a public response,"CITIBANK, N.A.",PA,151XX,Older American,Consent provided,Web,2015-03-27,Closed with explanation,Yes,No,1305453
2,2015-04-20,Bank account or service,Other bank product/service,"Making/receiving payments, sending money",NaN,XXXX 2015 : I called to make a payment on XXXX...,Company chooses not to provide a public response,U.S. BANCORP,PA,152XX,NaN,Consent provided,Web,2015-04-22,Closed with monetary relief,Yes,No,1337613
3,2013-04-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,NaN,NaN,JPMORGAN CHASE & CO.,VA,22406,Servicemember,NaN,Phone,2013-04-30,Closed with explanation,Yes,Yes,393900
4,2013-05-29,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",GA,30044,NaN,NaN,Referral,2013-05-31,Closed with explanation,Yes,No,418647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207255,2015-05-24,Debt collection,Credit card,Taking/threatening an illegal action,Sued w/o proper notification of suit,NaN,NaN,JPMORGAN CHASE & CO.,FL,33133,NaN,Consent not provided,Web,2015-05-24,Closed with explanation,Yes,No,1390395
207256,2012-01-10,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,JPMORGAN CHASE & CO.,NY,10312,NaN,NaN,Referral,2012-01-11,Closed without relief,Yes,Yes,12192
207257,2012-07-17,Student loan,Non-federal student loan,Repaying your loan,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",NH,032XX,NaN,NaN,Web,2012-07-18,Closed with explanation,Yes,No,118351
207258,2016-09-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,Near the end of XXXX 2016 I opened a Citigold ...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",CA,900XX,NaN,Consent provided,Web,2016-09-29,Closed with non-monetary relief,Yes,No,2138969


In [4]:
complaint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207260 entries, 0 to 207259
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 207260 non-null  object
 1   Product                       207260 non-null  object
 2   Sub-product                   164245 non-null  object
 3   Issue                         207260 non-null  object
 4   Sub-issue                     10347 non-null   object
 5   Consumer complaint narrative  29391 non-null   object
 6   Company public response       58458 non-null   object
 7   Company                       207260 non-null  object
 8   State                         205066 non-null  object
 9   ZIP code                      197974 non-null  object
 10  Tags                          28265 non-null   object
 11  Consumer consent provided?    51313 non-null   object
 12  Submitted via                 207260 non-null  object
 13 

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
import seaborn as sns
import statsmodels.api as sm
import pandas as pd
import numpy as np


In [23]:
X=complaint[['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via',  'Company response to consumer', 'Timely response?']]
X=pd.get_dummies(X)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(complaint['Consumer disputed?'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)


In [24]:
disputed_proportion = y_train.sum() / len(y_train)
disputed_proportion

0.21684719675769565

In [26]:
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler(random_state=123)
X_train, y_train = undersampler.fit_resample(X_train, y_train)
disputed_proportion = y_train.sum() / len(y_train)
disputed_proportion

0.5

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay


In [31]:
import xgboost as xgb

# Initialize the model
model_xgb = xgb.XGBClassifier(random_state=123)

# Train the model
model_xgb.fit(X_train, y_train)

# Make predictions (for evaluation)
y_pred = model_xgb.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

              precision    recall  f1-score   support

           0       0.84      0.53      0.65     32504
           1       0.27      0.63      0.38      8948

    accuracy                           0.55     41452
   macro avg       0.55      0.58      0.51     41452
weighted avg       0.71      0.55      0.59     41452

[[17157 15347]
 [ 3323  5625]]


In [13]:
32504*100+8948*600

8619200

In [32]:
17157*100+15347*190+3323*600+5625*190

7694180

In [40]:
# Assuming y_test contains your actual labels, with 1 indicating 'disputed'
cost_disputed = 600
cost_non_disputed = 100

# Calculating costs
total_cost_base_case = (y_test == 1).sum() * cost_disputed + (y_test == 0).sum() * cost_non_disputed
print(f"Base-Case Total Cost: ${total_cost_base_case}")


Base-Case Total Cost: $8619200


In [44]:

tn, fp, fn, tp = cm.ravel()

cost_extra_diligence = 90


total_cost_model = (tp + fp) * 190 + fn * cost_disputed + tn * cost_non_disputed
total_cost_model


7694180

In [39]:
def calculate_cost_with_threshold(y_true, y_prob, threshold):
    y_pred_adjusted = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred_adjusted)
    tn, fp, fn, tp = cm.ravel()
    total_cost = (tp + fp) * 190 + fn * 600 + tn * 100
    return total_cost


y_prob = model_xgb.predict_proba(X_test)[:, 1]

thresholds = np.linspace(0, 1, 101)
costs = [calculate_cost_with_threshold(y_test, y_prob, threshold) for threshold in thresholds]


min_cost = min(costs)
optimal_threshold = thresholds[costs.index(min_cost)]
print(f"Optimal Threshold: {optimal_threshold}, Minimum Total Cost: ${min_cost}")


Optimal Threshold: 0.41000000000000003, Minimum Total Cost: $7612170
